# Problem Set 3

For this problem set, you will expand on PS2 to perform and evaluate various sentiment classification methods.

Your name: Ben Hoffman

You abc123: Peo042

## Submission Instructions

After completing the exercises below, generate a pdf of the code **with** outputs. After that create a zip file containing both the completed exercise and the generated PDF/HTML. You are **required** to check the PDF/HTML to make sure all the code **and** outputs are clearly visible and easy to read. If your code goes off the page, you should reduce the line size. I generally recommend not going over 80 characters.

Finally, name the zip file using a combination of your the assigment and your name, e.g., ps3_rios.zip

## Exercise 1 (10 point)

For this step, you will load the **training dataset "twitdata_TEST.tsv"** and **test sentiment dataset "allTrainingData.tsv"**. The data should be loaded into 4 lists of strings: X_txt_train, X_txt_test, y_test, y_train.

- Note 1: when using csvreader, you need to pass the "quoting" the value csv.QUOTE_NONE.
- Note 2: the tweet is the 4th column (index=3) and the gold label (sentiment) is the 3rd column (index=2)
- Note 3: a tsv file is a tab-separated csv file

In [1]:
import csv

#open each file into a useable object
filetrain = list(csv.reader(open('allTrainingData.tsv'), delimiter ='\t', quoting = csv.QUOTE_NONE))
filetest = list(csv.reader(open('twitdata_TEST.tsv'), delimiter ='\t', quoting = csv.QUOTE_NONE))

#make empty lists for each variable
X_txt_train = []
y_train = []
X_txt_test = []
y_test = []

#loop over each file to isolate the needed information into the empty lists
for x in filetrain:
    X_txt_train.append(x[3])
    y_train.append(x[2])
    
for y in filetest:
    X_txt_test.append(y[3])
    y_test.append(y[2])


In [2]:
y_train[0]

'positive'

The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [3]:
assert(type(X_txt_train) == type(list()))
assert(type(X_txt_train[0]) == type(str()))
assert(type(X_txt_test) == type(list()))
assert(type(X_txt_test[0]) == type(str()))
assert(type(y_test) == type(list()))
assert(type(y_train) == type(list()))
assert(len(X_txt_test) == 3199)
assert(len(y_test) == 3199)
assert(len(X_txt_train) == 8018)
assert(len(y_train) == 8018)
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 2 (10 point)

This part is similar to HW2 (using the positive_words and negative_words variables). We will compare last homework's lexicon-based classification method with supervised models. Only make predictions on the test split and store all predictions in the list lex_test_preds. Next, calculate the **micro** precision, recall, and f1 scores using the lex_test_preds list.

- Note 1: use precision_score, recall_score, f1_score from sklearn.metrics with **average="micro"** to calculate the micro precision, ....

You can learn more about lexicon-based classification in Chapter 19.6. If you are interested, the chapter is available online for free at the following link: [Speech and Language Processing](https://web.stanford.edu/~jurafsky/slp3/19.pdf)

In [4]:
# DO NOT MODIFY THE CODE IN THIS CELL
class LexiconClassifier():
    def __init__(self):
        self.positive_words = set()
        with open('positive-words.txt', encoding = 'utf-8') as iFile:
            for row in iFile:
                self.positive_words.add(row.strip())

        self.negative_words = set()
        with open('negative-words.txt', encoding='iso-8859-1') as iFile:
            for row in iFile:
                self.negative_words.add(row.strip())

    def predict(self, sentence):
        num_pos_words = 0
        num_neg_words = 0
        for word in sentence.lower().split():
            if word in self.positive_words:
                num_pos_words += 1
            elif word in self.negative_words:
                num_neg_words += 1
        
        pred = 'neutral'        
        if num_pos_words > num_neg_words:
            pred = 'positive'
        elif num_pos_words < num_neg_words:
            pred = 'negative'
            
        return pred
    
    def count_pos_words(self, sentence):
        num_pos_words = 0
        for word in sentence.lower().split():
            if word in self.positive_words:
                num_pos_words += 1
        return num_pos_words

    def count_neg_words(self, sentence):
        num_neg_words = 0
        for word in sentence.lower().split():
            if word in self.negative_words:
                num_neg_words += 1
        return num_neg_words

In [5]:
# WRITE CODE HERE
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

#create a class variable using the premade code
C = LexiconClassifier()

#create an empty list to make hold the predicts
lex_test_preds = []

#loop over the test set using the predict feature of the class
for x in X_txt_test:
    lex_test_preds.append(C.predict(x))
    
#calculate model scores using y set and predicts above
precision = precision_score(y_test, lex_test_preds, average= 'micro')
recall = recall_score(y_test, lex_test_preds, average= 'micro')
f1 = f1_score(y_test, lex_test_preds, average= 'micro')
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1: {:.4f}".format(f1))

Precision: 0.5827
Recall: 0.5827
F1: 0.5827


The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [6]:
assert(type(lex_test_preds) == type(list()))
assert(type(lex_test_preds[0]) == type(str()))
assert(set(lex_test_preds) == set(["positive", "negative", "neutral"]))
assert(len(lex_test_preds) == len(y_test))
assert(type(precision) == type(float()) or type(precision) == type(np.float64()))
assert(type(recall) == type(float()) or type(recall) == type(np.float64()))
assert(type(f1) == type(float()) or type(f1) == type(np.float64()))
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 3 (20 point)

Again, using the LexiconClassifier, write code to generate a lists of lists where each sublist contains the number of positive words and negative words in a tweet. Assume we are give the train test datasets

``` python
X_txt_train = ["good good", "bad bad"]
X_txt_test = ["great", "bad bad great"]
```

you should write code that creates two lists of lists as follows:

``` python
X_train_lexicon_features = [[2, 0], [0,2]] # [2, 0] means the first tweeta has 2 positive words and 0 negative words.
X_test_lexicon_features = [[1, 0], [1, 2]]
```




In [7]:
# WRITE CODE HERE

#create empty lists to put lists into
X_train_lexicon_features = []
X_test_lexicon_features = []

#create class variable
lex = LexiconClassifier()

#determine number of pos and neg words
for x in X_txt_train:
    X_train_lexicon_features.append([lex.count_pos_words(x),lex.count_neg_words(x)])

for y in X_txt_test:
    X_test_lexicon_features.append([lex.count_pos_words(x),lex.count_neg_words(x)])

The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [8]:
assert(type(X_train_lexicon_features) == type(list()))
assert(type(X_test_lexicon_features) == type(list()))
assert(type(X_test_lexicon_features[0]) == type(list()))
assert(len(X_train_lexicon_features) == len(X_txt_train))
assert(len(X_test_lexicon_features) == len(X_txt_test))
assert(len(X_train_lexicon_features[0]) == 2)
assert(len(X_test_lexicon_features[0]) == 2)
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 4 (30 points)

For this task you should creat a feature matrix using CountVectorizer and train a LinearSVC model from scikit-learn. On the train split, use GridSearchCV to find the best LinearSVC C values (0.0001, 0.001, 0.001, 0.01, 0.1, 1, 10, or 100) based on the micro f1 scoring metric (hint: "micro" average) and set the cv parameter to 5. Also, with the CountVectorizer, only use unigrams (i.e., set ngram_range = (1,1)). Note that GridSearchCV will retrain the final classifier using the best parameters, so you don't need to do it manually.

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, f1_score

import numpy as np
np.random.seed(42)
import random
random.seed(42)

# WRITE CODE HERE

#make an empty vectorizer
Vec = CountVectorizer(ngram_range = (1,1))

#fit the training and transform the test sets
X_train = Vec.fit_transform(X_txt_train)
X_test = Vec.transform(X_txt_test)

#fit the model
linsvc = GridSearchCV(LinearSVC(), param_grid={'C': [.0001, 100]}, 
                      scoring = 'f1_micro', cv = 5)
linsvc.fit(X_train, y_train)

#find the best score
validation_score = linsvc.best_score_
print("Validation F1: {:.4f}".format(validation_score))


C:\Users\hoffm\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Validation F1: 0.6226


In [10]:
print(X_train.shape)
print(X_test.shape)

(8018, 20864)
(3199, 20864)


In [11]:
#make predictions using the model
svm_test_predictions = linsvc.predict(X_test)

#score the model
precision = precision_score(y_test, svm_test_predictions, average= 'micro')
recall = recall_score(y_test, svm_test_predictions, average= 'micro')
f1 = f1_score(y_test, svm_test_predictions, average= 'micro')
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1: {:.4f}".format(f1))

Precision: 0.6158
Recall: 0.6158
F1: 0.6158


The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [12]:
from scipy.sparse import csr_matrix
assert(type(X_train) == type(csr_matrix(0)) or type(X_train) == type(np.array(0)))
assert(type(X_test) == type(csr_matrix(0)) or type(X_test) == type(np.array(0)))
assert(X_train.shape[0] == len(X_txt_train))
assert(X_test.shape[0] == len(X_txt_test))
assert(X_train.shape[1] == X_test.shape[1])
assert(type(precision) == type(float()) or type(precision) == type(np.float64()))
assert(type(recall) == type(float()) or type(recall) == type(np.float64()))
assert(type(f1) == type(float()) or type(f1) == type(np.float64()))
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 5 (20 points)

Repeat the experiment from exercise 4, but include the lexicon features with the CountVectorizer features. Specifically, you need to concatenate the variables ```X_train_lexicon_features``` and ```X_test_lexicon_features``` with ```X_train``` and ```X_test```, respectively. Intuitively, we are performing feature engineering by adding "lexicon features".

HINT: You will need to convert the lexicon features to numpy arrays then call hstack from the scipy.sparse library (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.hstack.html)

In [13]:
import scipy.sparse as sp

import numpy as np
np.random.seed(42)
import random
random.seed(42)

# WRITE CODE HERE

#turn the variables into arrays
trainarray = np.array(X_train_lexicon_features)
testarray = np.array(X_test_lexicon_features)

#merge the two arrays horizontally
X_train_w_lex = sp.hstack([X_train, trainarray]).toarray()
X_test_w_lex = sp.hstack([X_test, testarray]).toarray()

print(X_train_w_lex.shape)
print(X_test_w_lex.shape)

(8018, 20866)
(3199, 20866)


In [14]:
#fit the model with the new variables
lexsvc = GridSearchCV(LinearSVC(tol = .0001), param_grid={'C': [.0001, 100]},
                      scoring = 'f1_micro', cv = 5)
lexsvc.fit(X_train_w_lex, y_train)

validation_score = lexsvc.best_score_
print("Validation F1: {:.4f}".format(validation_score))

svm_lex_test_predictions = lexsvc.predict(X_test_w_lex)

precision = precision_score(y_test, svm_lex_test_predictions, average= 'micro')
recall = recall_score(y_test, svm_lex_test_predictions, average= 'micro')
f1 = f1_score(y_test, svm_lex_test_predictions, average= 'micro')
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1: {:.4f}".format(f1))

C:\Users\hoffm\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\hoffm\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\hoffm\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\hoffm\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\hoffm\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

Validation F1: 0.6382
Precision: 0.5649
Recall: 0.5649
F1: 0.5649


The lines below give example inputs and correct outputs using asserts, and can be run to test the code. Passing these tests is necessary, but **NOT** sufficient to guarantee your implementation is correct. You may add additional test cases, but do not remove any tests.

In [15]:
from scipy.sparse import csr_matrix
assert(X_train_w_lex.shape[0] == len(X_txt_train))
assert(X_test.shape[0] == len(X_txt_test))
assert(X_train_w_lex.shape[1] == X_test.shape[1] + 2)
assert(X_train_w_lex.shape[1] == X_test_w_lex.shape[1])
assert(type(precision) == type(float()) or type(precision) == type(np.float64()))
assert(type(recall) == type(float()) or type(recall) == type(np.float64()))
assert(type(f1) == type(float()) or type(f1) == type(np.float64()))
print("Asserts Completed Successfully!")

Asserts Completed Successfully!


## Exercise 6 (10 points)

For this exercise, you will perform manual analysis of the predictions. Answer the questions below.

In [16]:
num_tweets = 0
for text, svm_pred, svm_lex_pred, lex_pred, y  in zip(X_txt_test, svm_test_predictions,
                                    svm_lex_test_predictions, lex_test_preds, y_test):
    print("Tweet: {}".format(text))
    print("Ground-Truth Class: {}".format(y))
    print("SVM Prediction: {}".format(svm_pred))
    print("SVM+Lexicon Prediction: {}".format(svm_lex_pred))
    print("Lexicon Model Prediction: {}".format(lex_pred))
    print()
    
    num_tweets += 1
    if num_tweets == 20:
        break

Tweet: Musical awareness: Great Big Beautiful Tomorrow has an ending, Now is the time does not
Ground-Truth Class: positive
SVM Prediction: positive
SVM+Lexicon Prediction: positive
Lexicon Model Prediction: positive

Tweet: On Radio786 100.4fm 7:10 Fri Oct 19 Labour analyst Shawn Hattingh: Cosatu's role in the context of unrest in the mining http://t.co/46pjzzl6
Ground-Truth Class: neutral
SVM Prediction: neutral
SVM+Lexicon Prediction: neutral
Lexicon Model Prediction: negative

Tweet: Kapan sih lo ngebuktiin,jan ngomong doang Susah Susah.usaha Aja blm udh nyerah,inget.if you never try you'll never know.cowok kok gentle bgt
Ground-Truth Class: negative
SVM Prediction: neutral
SVM+Lexicon Prediction: negative
Lexicon Model Prediction: positive

Tweet: Tomorrow come and hear @DavidWillettsMP&amp;@MASieghart debate "Navigating the new Higher Education market" 5.30pm, Jurys Inn #CPC12
Ground-Truth Class: neutral
SVM Prediction: positive
SVM+Lexicon Prediction: positive
Lexicon Model Pred

Complete the following tasks:
 
- Manually annotate all of the tweets printed above:
   1. Negative because it is saying the album does not have an ending
   1. Neutral
   1. Positive because it is saying you should try
   1. Neutral
   1. Neutral
   1. Positive
   1. Negative because it is saying the lack of update is preventing purchase
   1. Positive
   1. Neutral
   1. Positive
   1. Negative
   1. Neutral
   1. Positive
   1. Negative because it is about a mosque being set on fire
   1. Positive because it is meant to be humorous
   1. Neutral
   1. Positive
   1. Neutral because it is a mix of pos and neg sentences
   1. Neutral
   1. Positive

- How many of your annotations match the ground truth labels? Do you think the datasets labels are correct? (Use your intuition)
    - 15. Yes, for the most part. They match my annotations enough where I would trust the training model

- How many of your annotations match the lexicon-based model's predictions?
    - 9

- How many of your annotations match the SVM's predictions?
    - 12
    
- How many of your annotations match the SVM+Lexicon's predictions?
    - 11
    
- Do you see any major limitations of the linear SVM model? Use your intuition, I will accept most answers, as long as it makes some sense. Please describe and provide examples below:

The f scores are very low, which indicates some issues with the fit. I think that a linear relationship cannot be expressed using sentiment scores because there is not a set relationship between word and sentiment. For example, tweet 1 is about an album not having an ending, but taking the words as individuals makes the prediction positive. This is because the model places a certain linear coefficient on each word without looking at the meaning of the entire tweet.

## Extra Credit 1 (20 points)

For this extra credit the only goal is to improve your model on the test set (i.e., increase the micro f1 score). You may create new features, grid search over more parameters, try different feature weighting methods (e.g., TfidfVectorizer), or test different machine learning models. You can do whatever you want as long as the final test score improves, I will provide you with the extra credit points.

DO NOT TRAIN ON THE TEST SET. That is cheating!

In [17]:
# WRITE CODE HERE